In [ ]:
import logging

import numpy as np
import optimistix as optx

from atmodeller import (
    ChemicalSpecies,
    EquilibriumModel,
    Planet,
    SolverParameters,
    SpeciesNetwork,
    debug_logger,
)
from atmodeller.eos import get_eos_models
from atmodeller.solubility import get_solubility_models
from atmodeller.thermodata import IronWustiteBuffer

logger = debug_logger()
logger.setLevel(logging.INFO)
# For more output use DEBUG
# logger.setLevel(logging.DEBUG)

# Sub-Neptune (e.g. K2-18b) models from Bower et al. (2025)

The code blocks below must always be run, but then you can preferentially run only the models for ideal or real gases.

Parameters for the simulations

In [ ]:
number_of_realisations = 500
surface_temperature = 3000.0  # K

# For simulations with fixed mass and surface radius:
planet_mass = 5.154e25
surface_radius = 1.1225e7  # using M-R relation from Hakim+2018
mantle_melt_fraction = 1.0  # 0.1

RANDOM_SEED = 0
np.random.seed(RANDOM_SEED)

In [ ]:
solubility_models = get_solubility_models()

H2O_g = ChemicalSpecies.create_gas("H2O", solubility=solubility_models["H2O_basalt_dixon95"])
H2_g = ChemicalSpecies.create_gas("H2", solubility=solubility_models["H2_basalt_hirschmann12"])
O2_g = ChemicalSpecies.create_gas("O2")
CO_g = ChemicalSpecies.create_gas("CO", solubility=solubility_models["CO_basalt_yoshioka19"])
CO2_g = ChemicalSpecies.create_gas("CO2", solubility=solubility_models["CO2_basalt_dixon95"])
CH4_g = ChemicalSpecies.create_gas("CH4", solubility=solubility_models["CH4_basalt_ardia13"])

idealspecies_withsols = SpeciesNetwork((H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g))

In [ ]:
eos_models = get_eos_models()

H2O_rg = ChemicalSpecies.create_gas(
    "H2O",
    activity=eos_models["H2O_cork_holland98"],
    solubility=solubility_models["H2O_basalt_dixon95"],
)
H2_rg = ChemicalSpecies.create_gas(
    "H2",
    activity=eos_models["H2_chabrier21"],
    solubility=solubility_models["H2_basalt_hirschmann12"],
)
O2_rg = ChemicalSpecies.create_gas("O2")
CO_rg = ChemicalSpecies.create_gas(
    "CO",
    activity=eos_models["CO_cs_shi92"],
    solubility=solubility_models["CO_basalt_yoshioka19"],
)
CO2_rg = ChemicalSpecies.create_gas(
    "CO2",
    activity=eos_models["CO2_cs_shi92"],
    solubility=solubility_models["CO2_basalt_dixon95"],
)
CH4_rg = ChemicalSpecies.create_gas(
    "CH4",
    activity=eos_models["CH4_cs_shi92"],
    solubility=solubility_models["CH4_basalt_ardia13"],
)

realspecies_withsols = SpeciesNetwork((H2O_rg, H2_rg, O2_rg, CO_rg, CO2_rg, CH4_rg))

## Vary Hydrogen Mass Fraction

Hydrogen mass fraction varies from 0.1 to 3% of K2-18b's mass

In [ ]:
# Vary Linearly:

log10_H_frac = np.linspace(-1.0, 0.5, number_of_realisations)  # 0.1 to 3% of planet mass

# Fix Values for Linearly Varying Cases:
log10_ch_ratios = np.full(number_of_realisations, -0.5)  # 100X Solar
fO2_log10_shifts = np.full(number_of_realisations, -3)

h_kg = ((10**log10_H_frac) / 100) * planet_mass
c_kg = h_kg * 10**log10_ch_ratios

mass_constraints = {"H": h_kg, "C": c_kg}

In [ ]:
sub_neptune = Planet(
    temperature=surface_temperature,
    planet_mass=planet_mass,
    surface_radius=surface_radius,
    mantle_melt_fraction=mantle_melt_fraction,
)

### Ideal gas with solubilities

In [ ]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_ideal_varyHF_withsol = EquilibriumModel(idealspecies_withsols)

fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

model_ideal_varyHF_withsol.solve(
    state=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
)
output_ideal_varyHF_withsol = model_ideal_varyHF_withsol.output

# Write the complete solution to Excel
output_ideal_varyHF_withsol.to_excel("sub_neptune_ideal_withsol_varyHF")

# Write the data to a pickle file with dataframes
output_ideal_varyHF_withsol.to_pickle("sub_neptune_ideal_withsol_varyHF")

### Real gas with solubility

In [ ]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_real_varyHF_withsol = EquilibriumModel(realspecies_withsols)

fugacity_constraints = {O2_rg.name: IronWustiteBuffer(fO2_log10_shifts)}

model_real_varyHF_withsol.solve(
    state=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    # Use the ideal solution for the initial guess of the real case
    initial_log_number_moles=output_ideal_varyHF_withsol.log_number_moles,
    solver_parameters=solver_parameters,
)
output_real_varyHF_withsol = model_real_varyHF_withsol.output

# Write the complete solution to Excel
output_real_varyHF_withsol.to_excel("sub_neptune_real_withsol_varyHF")

# Write the data to a pickle file with dataframes
output_real_varyHF_withsol.to_pickle("sub_neptune_real_withsol_varyHF")

## Vary Oxygen Fugacity

fO2 varies from IW-6 to IW

In [ ]:
# Vary Linearly:
fO2_log10_shifts = np.linspace(-6, 0, number_of_realisations)  # IW-6 to IW


# Fix Values for Linearly Varying Cases:
log10_H_frac = np.full(number_of_realisations, 0)  # 1% of planet mass
log10_ch_ratios = np.full(number_of_realisations, -0.5)  # 100X Solar

h_kg = ((10**log10_H_frac) / 100) * planet_mass
c_kg = h_kg * 10**log10_ch_ratios

mass_constraints = {"H": h_kg, "C": c_kg}

In [ ]:
sub_neptune = Planet(
    temperature=surface_temperature,
    planet_mass=planet_mass,
    surface_radius=surface_radius,
    mantle_melt_fraction=mantle_melt_fraction,
)

### Ideal gas with solubility

In [ ]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_ideal_varyfO2_withsol = EquilibriumModel(idealspecies_withsols)

fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

model_ideal_varyfO2_withsol.solve(
    state=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
)
output_ideal_varyfO2_withsol = model_ideal_varyfO2_withsol.output

# Write the complete solution to Excel
output_ideal_varyfO2_withsol.to_excel("sub_neptune_ideal_withsol_varyfO2")

# Write the data to a pickle file with dataframes
output_ideal_varyfO2_withsol.to_pickle("sub_neptune_ideal_withsol_varyfO2")

### Real gas with solubility

In [ ]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_real_varyfO2_withsol = EquilibriumModel(realspecies_withsols)

fugacity_constraints = {O2_rg.name: IronWustiteBuffer(fO2_log10_shifts)}

model_real_varyfO2_withsol.solve(
    state=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
    # Use the ideal solution for the initial guess of the real case
    initial_log_number_moles=output_ideal_varyfO2_withsol.log_number_moles,
)
output_real_varyfO2_withsol = model_real_varyfO2_withsol.output

# Write the complete solution to Excel
output_real_varyfO2_withsol.to_excel("sub_neptune_real_withsol_varyfO2")

# Write the data to a pickle file with dataframes
output_real_varyfO2_withsol.to_pickle("sub_neptune_real_withsol_varyfO2")

## Vary C/H Ratio

C/H ratio varies from that of solar (log10(C/H) = -2.5) to bulk silicate Earth (log10(C/H)=0.1)

In [ ]:
# Vary Linearly:
log10_ch_ratios = np.linspace(-2.5, 0.1, number_of_realisations)  # Solar to BSE Ratios


# Fix Values for Linearly Varying Cases:
log10_H_frac = np.full(number_of_realisations, 0)  # 1% of planet mass
fO2_log10_shifts = np.full(number_of_realisations, -3)

h_kg = ((10**log10_H_frac) / 100) * planet_mass
c_kg = h_kg * 10**log10_ch_ratios

mass_constraints = {"H": h_kg, "C": c_kg}

In [ ]:
sub_neptune = Planet(
    temperature=surface_temperature,
    planet_mass=planet_mass,
    surface_radius=surface_radius,
    mantle_melt_fraction=mantle_melt_fraction,
)

### Ideal gas with solubility

In [ ]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_ideal_varyCtoH_withsol = EquilibriumModel(idealspecies_withsols)

fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

model_ideal_varyCtoH_withsol.solve(
    state=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
)
output_ideal_varyCtoH_withsol = model_ideal_varyCtoH_withsol.output

# Write the complete solution to Excel
output_ideal_varyCtoH_withsol.to_excel("sub_neptune_ideal_withsol_varyCtoH")

# Write the data to a pickle file with dataframes
output_ideal_varyCtoH_withsol.to_pickle("sub_neptune_ideal_withsol_varyCtoH")

### Real gas with solubility

In [ ]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_real_varyCtoH_withsol = EquilibriumModel(realspecies_withsols)

fugacity_constraints = {O2_rg.name: IronWustiteBuffer(fO2_log10_shifts)}

model_real_varyCtoH_withsol.solve(
    state=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
    # Use the ideal solution for the initial guess of the real case
    initial_log_number_moles=output_ideal_varyCtoH_withsol.log_number_moles,
)
output_real_varyCtoH_withsol = model_real_varyCtoH_withsol.output

# Write the complete solution to Excel
output_real_varyCtoH_withsol.to_excel("sub_neptune_real_withsol_varyCtoH")

# Write the data to a pickle file with dataframes
output_real_varyCtoH_withsol.to_pickle("sub_neptune_real_withsol_varyCtoH")

## Vary Planetary Surface Radius

Surface radius varies from 1.76 to 2.6 REarth, planet mass is fixed at 8.63 MEarth

In [ ]:
# For simulations with varying surface radius:
surface_radius = np.linspace(1.1225e7, 1.6647e7, number_of_realisations)  # Vary linearly

# Fix Values for Linearly Varying Cases:
log10_H_frac = np.full(
    number_of_realisations, 0.5
)  # ~3% of planet mass, used for fix Surf Radius and Planet Mass cases
log10_ch_ratios = np.full(number_of_realisations, -0.5)  # 100X Solar
fO2_log10_shifts = np.full(number_of_realisations, -3)

h_kg = ((10**log10_H_frac) / 100) * planet_mass
c_kg = h_kg * 10**log10_ch_ratios

mass_constraints = {"H": h_kg, "C": c_kg}

In [ ]:
sub_neptune = Planet(
    temperature=surface_temperature,
    planet_mass=planet_mass,
    surface_radius=surface_radius,
    mantle_melt_fraction=mantle_melt_fraction,
)

### Ideal gas with solubility

In [ ]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_ideal_varyRsurf_withsol = EquilibriumModel(idealspecies_withsols)

fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

model_ideal_varyRsurf_withsol.solve(
    state=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
)
output_ideal_varyRsurf_withsol = model_ideal_varyRsurf_withsol.output

# Write the complete solution to Excel
output_ideal_varyRsurf_withsol.to_excel("sub_neptune_ideal_withsol_varyRsurf")

# Write the data to a pickle file with dataframes
output_ideal_varyRsurf_withsol.to_pickle("sub_neptune_ideal_withsol_varyRsurf")

### Real gas with solubility

In [ ]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_real_varyRsurf_withsol = EquilibriumModel(realspecies_withsols)

fugacity_constraints = {O2_rg.name: IronWustiteBuffer(fO2_log10_shifts)}

model_real_varyRsurf_withsol.solve(
    state=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
    # Use the ideal solution for the initial guess of the real case
    initial_log_number_moles=output_ideal_varyRsurf_withsol.log_number_moles,
)
output_real_varyRsurf_withsol = model_real_varyRsurf_withsol.output

# Write the complete solution to Excel
output_real_varyRsurf_withsol.to_excel("sub_neptune_real_withsol_varyRsurf")

# Write the data to a pickle file with dataframes
output_real_varyRsurf_withsol.to_pickle("sub_neptune_real_withsol_varyRsurf")

## Vary Planetary Mass

Planet mass varies from 4 to 9 MEarth, surface radius is fixed at 1.76 REarth

In [ ]:
surface_radius = 1.1225e7  # using M-R relation from Hakim+2018


# For simulations with varying planet mass:

planet_mass_Earths = np.linspace(4, 9, number_of_realisations)  # Vary linearly from 4-9 MEarth
planet_mass = planet_mass_Earths * 5.9722e24


# Fix Values for Linearly Varying Cases:
log10_H_frac = np.full(
    number_of_realisations, 0.5
)  # ~3% of planet mass, used for fix Surf Radius and Planet Mass cases
log10_ch_ratios = np.full(number_of_realisations, -0.5)  # 100X Solar
fO2_log10_shifts = np.full(number_of_realisations, -3)

h_kg = ((10**log10_H_frac) / 100) * planet_mass
c_kg = h_kg * 10**log10_ch_ratios

mass_constraints = {"H": h_kg, "C": c_kg}

In [ ]:
sub_neptune = Planet(
    temperature=surface_temperature,
    planet_mass=planet_mass,
    surface_radius=surface_radius,
    mantle_melt_fraction=mantle_melt_fraction,
)

### Ideal gas with solubility

In [ ]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_ideal_varyMp_withsol = EquilibriumModel(idealspecies_withsols)

fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

model_ideal_varyMp_withsol.solve(
    state=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
)
output_ideal_varyMp_withsol = model_ideal_varyMp_withsol.output

# Write the complete solution to Excel
output_ideal_varyMp_withsol.to_excel("sub_neptune_ideal_withsol_varyMp")

# Write the data to a pickle file with dataframes
output_ideal_varyMp_withsol.to_pickle("sub_neptune_ideal_withsol_varyMp")

### Real gas with solubility

In [ ]:
solver = optx.LevenbergMarquardt
solver_parameters = SolverParameters(solver=solver)

model_real_varyMp_withsol = EquilibriumModel(realspecies_withsols)

fugacity_constraints = {O2_rg.name: IronWustiteBuffer(fO2_log10_shifts)}

model_real_varyMp_withsol.solve(
    state=sub_neptune,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
    solver_parameters=solver_parameters,
    # Use the ideal solution for the initial guess of the real case
    initial_log_number_moles=output_ideal_varyMp_withsol.log_number_moles,
)
output_real_varyMp_withsol = model_real_varyMp_withsol.output

# Write the complete solution to Excel
output_real_varyMp_withsol.to_excel("sub_neptune_real_withsol_varyMp")

# Write the data to a pickle file with dataframes
output_real_varyMp_withsol.to_pickle("sub_neptune_real_withsol_varyMp")